In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC

import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

In [2]:
dataDict = pickle.load(open('./data/pre_data/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict))

4722


In [3]:
keywordsDict = pickle.load(open('./data/pre_data/keywords_daum.pickled','rb'))

In [4]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [5]:
df_daum = pd.DataFrame.from_dict(dataDict, orient='index')
df_daum['date'] = pd.to_datetime(df_daum['date']).dt.date
df_daum.reset_index(inplace = True)
df_daum.rename(columns={'index':'id'}, inplace=True)
print (df_daum.shape)
df_daum.head()

(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{세보 효과, 공장, 남성들, 지역, 부작용, 연기, 건강}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{입냄새, 낮잠, 커피, 치아, 자세, 점심 식사, 철분, 디스크, 식후, 건강}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{진술, 주변, 경찰, 범행, 남편, 지인, 철원, 아들, 성관계, 앙심}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{혐의, 매장, 신영자 이사장, 롯데, 유통업체, 네이처리퍼블릭, 검찰, 징역}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{유치장, 혐의, 경찰, 과자, 범행, 폭행, 쪽지, 남자친구}"


In [11]:
textInfo = df_daum.title.values + df_daum.mainText.values

In [12]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [50]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    res = [x for x in words if len(x) != 1]
    return res

In [51]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [52]:
d2v_docs = [nav_tokenizer(x) for x in textInfo]
d2v_docs = [remove_stopwords(x, (stopwords)) for x in d2v_docs]

### word2vec

In [53]:
w2v_model = Word2Vec(d2v_docs, sg=1, size=3000,window=8, workers = 2, iter = 20)
w2v_model.init_sims(replace = True)

In [54]:
print(w2v_model.most_similar('신혜선'))

[('박시후', 0.7326027154922485), ('황금빛', 0.6782881021499634), ('내인생', 0.6689298748970032), ('서은수', 0.6599359512329102), ('나영희', 0.6090877056121826), ('셰어하우스', 0.5996896624565125), ('천호진', 0.5966459512710571), ('재벌가', 0.5924861431121826), ('서지안', 0.5906704664230347), ('소현경', 0.5865070819854736)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [55]:
print(w2v_model.most_similar('손흥민'))

[('토트넘', 0.6128614544868469), ('스토크시티', 0.5951695442199707), ('델레', 0.5923106074333191), ('에릭센', 0.584801197052002), ('득점포', 0.5789486765861511), ('왓퍼드', 0.5738787651062012), ('브라이튼', 0.5736539363861084), ('사우스햄턴', 0.5676836967468262), ('아포엘', 0.5526728630065918), ('슈팅력', 0.5514216423034668)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [56]:
print(w2v_model.most_similar('황금빛내인생'))

[('이다인', 0.6627808213233948), ('이진', 0.6580499410629272), ('나영희', 0.6489250063896179), ('대역', 0.6461194753646851), ('재벌남', 0.6439180970191956), ('빅픽처', 0.6356345415115356), ('류수영', 0.6262466907501221), ('정소영', 0.6260770559310913), ('김형석', 0.624772846698761), ('짝사랑', 0.6236057877540588)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [57]:
print(w2v_model.most_similar('홍준표'))

[('자유한국당', 0.6106137037277222), ('비박', 0.5419424772262573), ('이완구', 0.5327893495559692), ('비홍', 0.5286499261856079), ('김무성', 0.5214818716049194), ('정윤섭', 0.5191255211830139), ('하태경', 0.5152690410614014), ('샛문', 0.5146068334579468), ('주모', 0.5080316066741943), ('배훈식', 0.5031130313873291)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [58]:
print(w2v_model.most_similar('문재인'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('대통령', 0.5099797248840332), ('문재', 0.46571245789527893), ('국빈방문', 0.46407458186149597), ('조원진', 0.45894673466682434), ('헌재소장', 0.44682517647743225), ('방중', 0.4463379979133606), ('케어', 0.4389140009880066), ('대한애국당', 0.43745648860931396), ('조찬', 0.42287319898605347), ('서미선', 0.42001181840896606)]


In [59]:
print(w2v_model.most_similar('노무현'))

[('김대중', 0.604560136795044), ('김영삼', 0.6043766736984253), ('이회창', 0.577539324760437), ('박연차', 0.5461111664772034), ('장준하', 0.5172484517097473), ('서거', 0.509764552116394), ('현철', 0.5014920830726624), ('자유주의', 0.4971004128456116), ('박사모', 0.4930969178676605), ('대통령', 0.48949959874153137)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [60]:
print(w2v_model.most_similar('박근혜'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('탄핵', 0.5279452204704285), ('파면', 0.4856067895889282), ('탄핵안', 0.47975558042526245), ('헌재소장', 0.4633879065513611), ('이명박', 0.4533088207244873), ('비선실세', 0.4521252512931824), ('친박', 0.4476708769798279), ('국정농단', 0.44659578800201416), ('국정조사', 0.4457247853279114), ('결정문', 0.4396551251411438)]


In [61]:
print(w2v_model.most_similar('국정원'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('대선개입', 0.6240822076797485), ('추명호', 0.5867449045181274), ('특활비', 0.5730645656585693), ('화이트리스트', 0.5714545845985413), ('불법사찰', 0.5635067224502563), ('상납', 0.5563787817955017), ('이석수', 0.5449997186660767), ('국가정보원', 0.5396444797515869), ('원세훈', 0.5384938716888428), ('국가안보전략연구원', 0.5349069237709045)]


In [62]:
print(w2v_model.most_similar('이낙연'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('국무총리', 0.7034814953804016), ('정부서울청사', 0.6465581059455872), ('소방점검', 0.6225780844688416), ('우원식', 0.6045769453048706), ('밝혀내', 0.5869120359420776), ('서울시장', 0.5816628932952881), ('국화꽃', 0.5807350873947144), ('대형화재', 0.5757453441619873), ('진상조사', 0.5702327489852905), ('눈총', 0.5576291084289551)]


In [63]:
print(w2v_model.most_similar('이국종'))

[('중증외상센터', 0.7223384976387024), ('아주대병원', 0.7162899971008301), ('문종환', 0.6374176740646362), ('외상외과', 0.6255048513412476), ('오청', 0.6252435445785522), ('귀순병', 0.6228920817375183), ('피눈물', 0.6043981909751892), ('총상', 0.5979452133178711), ('권준식', 0.5955302119255066), ('아주대', 0.5900964736938477)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [64]:
print(w2v_model.most_similar('김성태'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('원내대표', 0.6414122581481934), ('정우택', 0.5313476324081421), ('예결위', 0.5116093754768372), ('국화꽃', 0.49200814962387085), ('야합', 0.4799313545227051), ('정윤섭', 0.47275856137275696), ('운영위', 0.46262991428375244), ('한선교', 0.4583285450935364), ('우원식', 0.45710790157318115), ('본청', 0.4562428593635559)]


In [65]:
print(w2v_model.most_similar('더불어민주당'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('우원식', 0.6339163184165955), ('추미애', 0.632307231426239), ('안민석', 0.5760869979858398), ('간사인', 0.5600768327713013), ('오신환', 0.5508747100830078), ('정우택', 0.5487918853759766), ('법인세법', 0.5481700301170349), ('운영위', 0.5452384948730469), ('이철희', 0.5369426012039185), ('응진', 0.5358690619468689)]


In [66]:
print(w2v_model.most_similar('새누리당'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('서청원', 0.6107808351516724), ('한나라당', 0.5803678631782532), ('시사저널', 0.5662543773651123), ('이혜훈', 0.5487267374992371), ('홍문종', 0.5435687303543091), ('김세연', 0.5435288548469543), ('이수담', 0.5431239008903503), ('당대표', 0.5408294200897217), ('탈당', 0.5365853309631348), ('경기지사', 0.5353192090988159)]


In [67]:
print(w2v_model.most_similar('자유한국당'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('홍준표', 0.6106137037277222), ('한국당', 0.5365824699401855), ('한선교', 0.5110212564468384), ('추미애', 0.5076382160186768), ('원내대표', 0.5065729022026062), ('하태경', 0.4927017390727997), ('정우택', 0.4898712933063507), ('정윤섭', 0.48840010166168213), ('김무성', 0.485804945230484), ('오신환', 0.4855045974254608)]


In [68]:
print(w2v_model.most_similar('국민의당'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('안철수', 0.7005809545516968), ('천정배', 0.6164925694465637), ('반대파', 0.5918814539909363), ('창당', 0.5834498405456543), ('유성엽', 0.5834038257598877), ('중진', 0.5833162069320679), ('의석수', 0.5822614431381226), ('바른정당', 0.5813928842544556), ('양당', 0.5791914463043213), ('야합', 0.5680350065231323)]


In [69]:
print(w2v_model.most_similar('안철수'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('국민의당', 0.7005808353424072), ('반대파', 0.6289240121841431), ('간신배', 0.6281692981719971), ('바른정당', 0.5961863398551941), ('창당', 0.5944721102714539), ('합당', 0.5907137393951416), ('유승민', 0.5876875519752502), ('중앙당', 0.5828919410705566), ('컨설턴트', 0.5821088552474976), ('대표직', 0.5809271931648254)]


In [70]:
print(w2v_model.most_similar('유승민'))

[('바른정당', 0.7132652997970581), ('안철수', 0.587687611579895), ('적합도', 0.571022093296051), ('통합신당', 0.5699814558029175), ('박영선', 0.5631622672080994), ('김세연', 0.5615983605384827), ('이학재', 0.551135778427124), ('야권', 0.5479469895362854), ('컨설턴트', 0.5457814335823059), ('샛문', 0.5410115122795105)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [71]:
print(w2v_model.most_similar('박지성'))

[('긱스', 0.5702229738235474), ('차범근', 0.5432595610618591), ('트래', 0.49573448300361633), ('서포터즈', 0.4834112226963043), ('트라', 0.47805237770080566), ('김상열', 0.46404698491096497), ('홍명보', 0.4624532461166382), ('유스', 0.46015551686286926), ('네빌', 0.4571748375892639), ('전달식', 0.45695728063583374)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [72]:
w2v_model.save('./data/pre_data/word2vec_category_size3000_window8_iter10_by_mecab.w2v')

In [73]:
embedding_matrix = np.zeros((len(w2v_model.wv.vocab), 3000))
for i in range(len(w2v_model.wv.vocab)):
    embedding_vector = w2v_model.wv[w2v_model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [74]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, embeddings, merge

In [75]:
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# input words - in this case we do sample by sample evaluations of the similarity
valid_word = Input((1,), dtype='int32')
other_word = Input((1,), dtype='int32')
# setup the embedding layer
embeddings = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                      weights=[embedding_matrix])
embedded_a = embeddings(valid_word)
embedded_b = embeddings(other_word)
similarity = merge([embedded_a, embedded_b], mode='cos', dot_axes=2)
# create the Keras model
k_model = Model(input=[valid_word, other_word], output=similarity)

def get_sim(valid_word_idx, vocab_size):
    sim = np.zeros((vocab_size,))
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    in_arr1[0,] = valid_word_idx
    for i in range(vocab_size):
        in_arr2[0,] = i
        out = k_model.predict_on_batch([in_arr1, in_arr2])
        sim[i] = out
    return sim

# now run the model and get the closest words to the valid examples
for i in range(valid_size):
    valid_word = w2v_model.wv.index2word[valid_examples[i]]
    top_k = 8  # number of nearest neighbors
    sim = get_sim(valid_examples[i], len(w2v_model.wv.vocab))
    nearest = (-sim).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in range(top_k):
        close_word = w2v_model.wv.index2word[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("me...)`
  


Nearest to 선수: 주전급, 외국인선수, 호령, 김경언, 김정혁, 정재훈, 휘문고, 강영식,
Nearest to 올해: 선수상, 조정훈, 신재영, 3관왕, 퓨쳐스리그, 전관왕, 임정우, 손승락,
Nearest to 이야기: 들려주, 박문수, 진시황, 사석, 쑥스럽, 스릴러, 어사, 이승,
Nearest to 일본: 도쿄, 스포츠호치, 지바, 바히드, 니치, 스포츠파크, 윤덕여, 할릴호지치,
Nearest to 사이: 월화드라마, 김진원, 버사, 원진아, 쇼핑몰, 강두, 박시은, 스며들,
Nearest to 사실: 진술서, 증거자료, 실토, 친자, 차명, 혐의점, 강순덕, 명예훼손죄,
Nearest to 대표: 홍준표, 대표직, 안철수, 창당, 하태경, 송아, 친안, 인재,
Nearest to 투수: 필승조, 좌완, 불펜, 임찬규, 배영수, 좌완투수, 함덕주, 피어밴드,
Nearest to 오후: 인터콘티넨탈호텔, 메이플홀, 여의도동, 4시, 하동, 입관식, 고양시, 임헌정,
Nearest to 기자: 조현, 아나, 엄기찬, 임창, 다희, 정형근, 설희, 최은지,
Nearest to 롯데: 자이언츠, 린드블럼, 민병헌, 강민호, 사도스키, 윤길현, 브룩스, 레일리,
Nearest to 친구: 치맥, 페트리, 로빈, 놀이동산, 문상돈, 창덕궁, 코펜하겐, 유람선,
Nearest to 관계자: 내막, 다각도, 성산읍, 플럼, 확답, 서울시교육청, 시공업체, 광역수사대,
Nearest to 공개: 허민, 축가, 홈즈, 절친, 빽가, 윌엔터테인먼트, 강연자, 포로수용소,
Nearest to 때문: 싫어하, 카테터, 덜하, 두뇌, 졸음, 합병증, 내키, 제트기류,
Nearest to 지난해: 영업이익, 뒷걸음질, 실질소득, 출고량, 선두주자, 국세, 수출액, 임정우,


In [ ]:
w2v_model.wv.